In [1]:
import os
import sys
import absl.logging
import PIL.Image

import numpy as np
import pandas as pd

from typing import Callable
from tensorflow import keras

module_path = os.path.abspath(os.path.join('..', '..', '..'))

if module_path not in sys.path:
    sys.path.append(os.path.join(module_path))

from functions.ciou import ciou_loss, ciou_metric
from functions.loading_data import SMALLER_HEIGHT, SMALLER_WIDTH
from functions.model_running import get_run_number, finalize_run

absl.logging.set_verbosity(absl.logging.ERROR)

In [2]:
get_names = lambda root_path: [
    file_name.split('.')[0]
    for dir_path, _, file_names in os.walk(root_path)
    for file_name in file_names
]
get_paths = lambda path: [f'{os.path.join(root, file)}' for root, dirs, files in os.walk(path) for file in files]
base_dir = os.path.join('..', '..', '..', '..', '..', 'data1', 'images_original_inception_resnet_v2_200x150_splitted')
train_dir = os.path.join(base_dir, 'training')
valid_dir = os.path.join(base_dir, 'validation')

In [3]:
def get_images_array(paths: list[str]) -> np.ndarray:
    rows = []
    rescale = keras.layers.Rescaling(1./255)

    for path in paths:
        with PIL.Image.open(path) as image:
            image_array = np.asarray(image)
            rescaled_image = rescale(image_array)
            rows.append(rescaled_image)

    return np.array(rows)


train_paths = get_paths(train_dir)
valid_paths = get_paths(valid_dir)
X_train = get_images_array(train_paths)
X_valid = get_images_array(valid_paths)

In [4]:
train_names = set(get_names(train_dir))
valid_names = set(get_names(valid_dir))
metadata_path = os.path.join('..', '..', '..', '..', '..', 'data1', 'HAM10000_metadata_ext.csv')
data = pd.read_csv(metadata_path).sort_values(by='image_id')
relevant_cols = ['top', 'left', 'bottom', 'right']
train_df = data[data['image_id'].isin(train_names)][relevant_cols]
valid_df = data[data['image_id'].isin(valid_names)][relevant_cols]
ys_train = train_df.to_numpy().astype(float)
ys_train[:, [0, 2]] /= SMALLER_HEIGHT
ys_train[:, [1, 3]] /= SMALLER_WIDTH
ys_valid = valid_df.to_numpy().astype(float)
ys_valid[:, [0, 2]] /= SMALLER_HEIGHT
ys_valid[:, [1, 3]] /= SMALLER_WIDTH

In [5]:
def get_attentive_model(loss: Callable, metric: Callable) -> keras.Model:
    def get_branch(_input: keras.layers.Layer, kernel_size: int) -> keras.layers.Layer:
        conv = keras.layers.Conv2D(128, kernel_size, padding='same', strides=2, activation='relu')(_input)
        conv = keras.layers.MaxPooling2D()(conv)

        return conv

    def get_inner_conv_module(prev: keras.layers.Layer, filters: int) -> keras.layers.Layer:
        conv_module = keras.layers.Conv2D(filters, 3, padding='same', activation='relu')(prev)
        conv_module = keras.layers.MaxPooling2D()(conv_module)
        conv_module = keras.layers.Dropout(.4)(conv_module)

        return conv_module

    def get_outer_conv_module(prev: keras.layers.Layer) -> keras.layers.Layer:
        conv_module = get_inner_conv_module(prev, 64)
        conv_module = get_inner_conv_module(conv_module, 128)
        conv_module = get_inner_conv_module(conv_module, 256)

        return conv_module

    def get_attention_module(prev: keras.layers.Layer, num: int) -> keras.layers.Layer:
        gap_layer = keras.layers.GlobalAveragePooling2D(name=f'attention_gap_{num}')(prev)
        gap_layer_res = keras.layers.Reshape((1, 1, 256))(gap_layer)
        dense = keras.layers.Dense(256, activation='relu')(gap_layer_res)
        dense = keras.layers.Dense(256, activation='softmax')(dense)
        mul_layer = keras.layers.Multiply()([prev, dense])

        return mul_layer

    def get_locator_module(prev: keras.layers.Layer) -> keras.layers.Layer:
        locator_module = keras.layers.Flatten()(prev)
        locator_module = keras.layers.Dense(256, activation='relu')(locator_module)
        locator_module = keras.layers.Dense(128, activation='relu')(locator_module)

        return locator_module

    _input = keras.layers.Input(shape=(SMALLER_HEIGHT, SMALLER_WIDTH, 3))
    branch1 = get_branch(_input, 3)
    branch2 = get_branch(_input, 5)
    branch3 = get_branch(_input, 7)
    merged_branches = keras.layers.concatenate([branch1, branch2, branch3])
    conv_module = get_outer_conv_module(merged_branches)
    attention_module1 = get_attention_module(conv_module, 1)
    attention_module2 = get_attention_module(conv_module, 2)
    merged_attentions = keras.layers.concatenate([attention_module1, attention_module2])
    locator_module = get_locator_module(merged_attentions)
    output = keras.layers.Dense(4, activation='sigmoid', name='root')(locator_module)
    gap_attention1 = keras.layers.GlobalAveragePooling2D()(attention_module1)
    gap_attention2 = keras.layers.GlobalAveragePooling2D()(attention_module2)
    aux_output = keras.layers.Dot(axes=1, normalize=True, name='dot')([gap_attention1, gap_attention2])
    model = keras.Model(_input, outputs=[output, aux_output])

    model.compile(
        optimizer='adam',
        loss={
            'root': loss,
            'dot': keras.losses.mean_squared_error,
        },
        loss_weights={'root': 1.0, 'dot': 0.1},
        metrics={
            'root': metric,
            'dot': keras.metrics.mean_squared_error
        })

    print(model.summary())

    return model

In [6]:
def run_multi_attentive_model(
        model_factory: Callable,
        model_name: str,
        loss: Callable,
        metric: Callable,
        reduction_patience=5,
        monitor='val_root_ciou_metric'):
    MIN_DELTA = .001
    early_stopping = keras.callbacks.EarlyStopping(
        monitor=monitor,
        mode='max',
        patience=20,
        min_delta=MIN_DELTA)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor=monitor,
        mode='max',
        factor=0.95,
        min_delta=MIN_DELTA,
        patience=reduction_patience,
        min_lr=0.0005,
        verbose=1)
    model_checkpoint = keras.callbacks.ModelCheckpoint(
        filepath=os.path.join('..', '..', '..', '..', 'tmp_models', model_name + '_{epoch}'),
        save_best_only=True)
    tensor_board = keras.callbacks.TensorBoard(log_dir=os.path.join('..', '..', '..', '..', 'tensor_logs', model_name))
    model = model_factory(loss, metric)

    return model.fit(
        X_train,
        {'root': ys_train, 'dot': np.zeros((len(ys_train,),))},
        validation_data=(X_valid, {'root': ys_valid, 'dot': np.zeros((len(ys_valid,),))}),
        epochs=500,
        batch_size=64,
        callbacks=[reduce_lr, model_checkpoint, tensor_board, early_stopping])

In [7]:
model_base_name = 'custom_models_7_multiattentive_model'
run_number = get_run_number(model_base_name)
model_name = f'{model_base_name}_{run_number}'
history = run_multi_attentive_model(get_attentive_model, model_name, ciou_loss, ciou_metric)
ROOT = os.path.join('..', '..', '..', '..')
DS_NAME = 'data1_roi'
plot_name = f'{model_name}.pdf'

finalize_run(ROOT, plot_name, model_base_name, DS_NAME, history, plot_mode = None)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 75, 100, 128  3584        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 75, 100, 128  9728        ['input_1[0][0]']                
                                )                                                             

INFO:tensorflow:Assets written to: ..\..\..\..\tmp_models\custom_models_7_multiattentive_model_20_1\assets


126/126 [==============================] - 26s 154ms/step - loss: 0.5330 - root_loss: 0.5322 - dot_loss: 0.0087 - root_ciou_metric: 0.4674 - dot_mean_squared_error: 0.0087 - val_loss: 0.5104 - val_root_loss: 0.5104 - val_dot_loss: 2.0053e-17 - val_root_ciou_metric: 0.4894 - val_dot_mean_squared_error: 2.0053e-17 - lr: 0.0010
Epoch 2/500
126/126 [==============================] - 15s 116ms/step - loss: 0.5048 - root_loss: 0.5048 - dot_loss: 6.2099e-21 - root_ciou_metric: 0.4956 - dot_mean_squared_error: 6.2099e-21 - val_loss: 0.5278 - val_root_loss: 0.5278 - val_dot_loss: 8.8223e-13 - val_root_ciou_metric: 0.4720 - val_dot_mean_squared_error: 8.8223e-13 - lr: 0.0010
Epoch 3/500
126/126 [==============================] - 14s 115ms/step - loss: 0.5000 - root_loss: 0.5000 - dot_loss: 1.2545e-16 - root_ciou_metric: 0.5004 - dot_mean_squared_error: 1.2545e-16 - val_loss: 0.5475 - val_root_loss: 0.5475 - val_dot_loss: 6.9213e-09 - val_root_ciou_metric: 0.4526 - val_dot_mean_squared_error: 6.9

INFO:tensorflow:Assets written to: ..\..\..\..\tmp_models\custom_models_7_multiattentive_model_20_4\assets


126/126 [==============================] - 17s 135ms/step - loss: 0.4952 - root_loss: 0.4952 - dot_loss: 3.8996e-11 - root_ciou_metric: 0.5048 - dot_mean_squared_error: 3.8996e-11 - val_loss: 0.5039 - val_root_loss: 0.5039 - val_dot_loss: 6.4809e-06 - val_root_ciou_metric: 0.4961 - val_dot_mean_squared_error: 6.4809e-06 - lr: 0.0010
Epoch 5/500
125/126 [============================>.] - ETA: 0s - loss: 0.4923 - root_loss: 0.4923 - dot_loss: 6.1692e-11 - root_ciou_metric: 0.5077 - dot_mean_squared_error: 6.1692e-11INFO:tensorflow:Assets written to: ..\..\..\..\tmp_models\custom_models_7_multiattentive_model_20_5\assets


...

126/126 [==============================] - 17s 132ms/step - loss: 0.4908 - root_loss: 0.4908 - dot_loss: 4.6256e-05 - root_ciou_metric: 0.5096 - dot_mean_squared_error: 4.6256e-05 - val_loss: 0.4949 - val_root_loss: 0.4949 - val_dot_loss: 0.0000e+00 - val_root_ciou_metric: 0.5049 - val_dot_mean_squared_error: 0.0000e+00 - lr: 9.0250e-04
Epoch 17/500
126/126 [==============================] - 14s 111ms/step - loss: 0.4907 - root_loss: 0.4907 - dot_loss: 0.0000e+00 - root_ciou_metric: 0.5099 - dot_mean_squared_error: 0.0000e+00 - val_loss: 0.4956 - val_root_loss: 0.4956 - val_dot_loss: 0.0000e+00 - val_root_ciou_metric: 0.5040 - val_dot_mean_squared_error: 0.0000e+00 - lr: 9.0250e-04
Epoch 18/500
126/126 [==============================] - 14s 110ms/step - loss: 0.4909 - root_loss: 0.4909 - dot_loss: 0.0000e+00 - root_ciou_metric: 0.5096 - dot_mean_squared_error: 0.0000e+00 - val_loss: 0.4950 - val_root_loss: 0.4950 - val_dot_loss: 0.0000e+00 - val_root_ciou_metric: 0.5047 - val_dot_mean_

INFO:tensorflow:Assets written to: ..\..\..\..\tmp_models\custom_models_7_multiattentive_model_20_19\assets


126/126 [==============================] - 17s 132ms/step - loss: 0.4908 - root_loss: 0.4908 - dot_loss: 0.0000e+00 - root_ciou_metric: 0.5098 - dot_mean_squared_error: 0.0000e+00 - val_loss: 0.4947 - val_root_loss: 0.4947 - val_dot_loss: 0.0000e+00 - val_root_ciou_metric: 0.5051 - val_dot_mean_squared_error: 0.0000e+00 - lr: 9.0250e-04
Epoch 20/500
125/126 [============================>.] - ETA: 0s - loss: 0.4908 - root_loss: 0.4908 - dot_loss: 0.0000e+00 - root_ciou_metric: 0.5092 - dot_mean_squared_error: 0.0000e+00
Epoch 20: ReduceLROnPlateau reducing learning rate to 0.0008573750033974647.
126/126 [==============================] - 14s 110ms/step - loss: 0.4907 - root_loss: 0.4907 - dot_loss: 0.0000e+00 - root_ciou_metric: 0.5096 - dot_mean_squared_error: 0.0000e+00 - val_loss: 0.4955 - val_root_loss: 0.4955 - val_dot_loss: 0.0000e+00 - val_root_ciou_metric: 0.5041 - val_dot_mean_squared_error: 0.0000e+00 - lr: 9.0250e-04
Epoch 21/500
126/126 [==============================] - 14s

INFO:tensorflow:Assets written to: ..\..\..\..\tmp_models\custom_models_7_multiattentive_model_20_24\assets


126/126 [==============================] - 16s 131ms/step - loss: 0.4909 - root_loss: 0.4909 - dot_loss: 0.0000e+00 - root_ciou_metric: 0.5093 - dot_mean_squared_error: 0.0000e+00 - val_loss: 0.4946 - val_root_loss: 0.4946 - val_dot_loss: 0.0000e+00 - val_root_ciou_metric: 0.5052 - val_dot_mean_squared_error: 0.0000e+00 - lr: 8.5737e-04
Epoch 25/500
125/126 [============================>.] - ETA: 0s - loss: 0.4907 - root_loss: 0.4907 - dot_loss: 0.0000e+00 - root_ciou_metric: 0.5093 - dot_mean_squared_error: 0.0000e+00
Epoch 25: ReduceLROnPlateau reducing learning rate to 0.0008145062311086804.
126/126 [==============================] - 14s 110ms/step - loss: 0.4908 - root_loss: 0.4908 - dot_loss: 0.0000e+00 - root_ciou_metric: 0.5089 - dot_mean_squared_error: 0.0000e+00 - val_loss: 0.4947 - val_root_loss: 0.4947 - val_dot_loss: 0.0000e+00 - val_root_ciou_metric: 0.5051 - val_dot_mean_squared_error: 0.0000e+00 - lr: 8.5737e-04
